# Patent Analysis

In [1]:
%%capture
# !pip install -r requirements.txt

## Importing Libraries

In [23]:
%%capture
import sys
import importlib
sys.path.append('src')


import preprocess_data
import qaagent

importlib.reload(preprocess_data)
importlib.reload(qaagent)


In [6]:

import nltk
nltk.download('all', quiet=True)

True

## Downloading patents and preprocessing

In [15]:
year = 2023
month = 1
day = 5

In [16]:
saved_patent_names = preprocess_data.parse_and_save_patents(year, month, day, True)

### Downloading weekly patent files...
Building the URL...
URL constructed: https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/ipa230105.zip
Requesting the file...
File retrieved successfully. Starting download...
File downloaded successfully. Starting extraction...
File extracted successfully.
ZIP file c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\patents.zip deleted after extraction.
### Extracting individual patents...
Locating the patent file...
Reading the patent file...
Splitting the XMl file into individual XMLs...
Total patents found: 7823
Creating directory to store individual patents...
Writing individual patents to separate txt files...
Patent 20230000001 does not belong to section 'C'. Skipping this patent.
Patent 20230000002 does not belong to section 'C'. Skipping this patent.
Patent 20230000003 does not belong to section 'C'. Skipping this patent.
Patent 20230000004 does not belong to section 'C'. Skipping this patent.
Patent 202300000

## Loading data and querying sample file


In [17]:
Prompt = """
Carefully review the given context and extract as much information as possible about measurements, their values, and units mentioned in the context. 

For each measurement, please provide the following details:
- The context in which the measurement is mentioned. (Measurement Context)
- The specific value or range given for the measurement. (Measured Value)
- The unit of the measurement, if provided. (Unit)
- What the measurement is referring to, if it can be determined from the context (e.g., a physical property, a characteristic, a specific part of a device, etc.). (Refers To)


Format your response in a structured JSON-like format, as follows:

{
  "Content": [
    {
      "Measurement Context": "context",
      "Measured Value": "value",
      "Unit": "unit",
      "Refers To": "reference"
    },
    // ... additional measurements, if present
  ]
}

If multiple measurements are present in the text, each should be listed as a separate object within the "Content" array.

Try to provide as complete and accurate information as possible. If certain details cannot be determined, indicate this by writing "Unknown".

Do not print any response other than the json.

Try to close json if the response more than maximum token.
"""



In [18]:
import json
import os

# Creating an empty dictionary to store the JSON data
json_files = {}

# Specifying the path to the JSON files
folder_path = os.path.join(
        os.getcwd(),
        "output"
    )


# Iterating through every file in the directory
for file_name in os.listdir(folder_path):
    # Checking if the current file is a JSON file
    if file_name.endswith('.json'):
        # Defining the path to the file
        file_path = os.path.join(folder_path, file_name)

        # Opening the JSON file
        with open(file_path, 'r') as file:
            # Loading the JSON data from the file
            data = json.load(file)

            # Adding the JSON data to the dictionary
            # The key is the file name split at the first "-" and taking the first part
            key_name = file_name.split('-')[0]
            json_files[key_name] = data


# Printing the dictionary to check if the data was loaded correctly
poc = []
for key in json_files:
    print(f"{key}: {json_files[key]}")
    poc.append(key)



US20230001042A1: {'Content': [{'Measurement Context': 'Atomic resolution high-angle annular dark field scanning transmission electron microscopy (HAADF-STEM) imaging was performed on a JEOL Grand ARM300CF microscope with double aberration correctors at 300 kV.', 'Measured Value': '300', 'Unit': 'kV', 'Refers To': 'Voltage of microscope'}, {'Measurement Context': 'Images were collected after ex-situ calcination of samples at 450° C.', 'Measured Value': '450', 'Unit': '°C', 'Refers To': 'Temperature for calcination of samples'}, {'Measurement Context': 'using a convergence semiangle of 22 mrad, and inner- and outercollection angles of 83 and 165 mrad, respectively.', 'Measured Value': '22, 83, 165', 'Unit': 'mrad', 'Refers To': 'Convergence semiangle and inner- and outercollection angles in microscopy'}, {'Measurement Context': 'A beam current of 11 pA was used with pixel time of 4 μs.', 'Measured Value': '11, 4', 'Unit': 'pA, μs', 'Refers To': 'Beam current and pixel time in microscopy'

In [19]:
indices = []
for poc_item in poc:
    for i, patent_path in enumerate(saved_patent_names):
        if poc_item in patent_path:
            indices.append(i)
print(indices)


[96, 130, 227, 497, 502, 503, 507, 602]


In [31]:
raw, output = qaagent.call_TA_to_json(Prompt, year, month, day, saved_patent_names, count=507)

Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230002559A1-20230105.XML.txt
Running Analyze Document chain...


KeyboardInterrupt: 

In [26]:
for idx in indices:
    raw, output = qaagent.call_TA_to_json(Prompt, year, month, day, saved_patent_names, count=idx)
    print(raw)

Loading documents from: c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105\US20230001042A1-20230105.XML.txt
Running Analyze Document chain...
{
  "Content": [
    {
      "Measurement Context": "the cover layer is formed to have a weight of equal to or greater than 0.05% and less than 0.3% with respect to a total weight of a filament forming each ultrafine thread",
      "Measured Value": "0.05% - 0.3%",
      "Unit": "%",
      "Refers To": "weight of the cover layer"
    },
    {
      "Measurement Context": "the cover layer is formed to have a weight of equal to or greater than 0.1% and less than 0.3% with respect to a total weight of a filament forming each ultrafine thread",
      "Measured Value": "0.1% - 0.3%",
      "Unit": "%",
      "Refers To": "weight of the cover layer"
    },
    {
      "Measurement Context": "the core thread 110 is, in the present embodiment, formed to have a thickness of about",
      "Measured Value": "0.27",
      "Unit": "mm",
   

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


KeyboardInterrupt: 

## Updating requirments file

In [ ]:
# !pip freeze > requirements.txt